In [68]:
MODEL = "gpt-4o-2024-08-06"
#MODEL = "gpt-4o-mini"
#MODEL = "gpt-3.5-turbo"
#prompt = [{"role": "user", "content": ""}]
loops = 1
#temperature = 1.5
#file = f"{MODEL}/single message/2" #alphabet/5"
file = "best chat"

import os
from openai import OpenAI
import pandas as pd
from IPython.display import display_markdown
import copy

client = OpenAI()
df_path = os.getcwd()
chat = False


def transform_roles(chat,assistant_transform_type = "user-assistant"):
    prompt_messages = copy.deepcopy(chat)
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages) and prompt_messages[-i]["role"] != "system":
        if i%2 == 1:
          prompt_messages[-i]["role"] = "user"
        else:
          prompt_messages[-i]["role"] = "assistant"
        i += 1
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type
    return prompt_messages
def load (path):
    with open(path, "r") as f:
        completions = pd.read_json(f, lines=True)
    return completions

In [12]:
#completions = load("completions/gpt-3.5-turbo/alphabet/5")
#completions

In [36]:
# Load completions from json, if continuing with a file generated in the past

try:
    with open(os.path.join(df_path,file), "r") as f:
        completions = pd.read_json(f, lines=True)

# If not, create a new completions database

except FileNotFoundError:
    completions = pd.DataFrame([],columns=['Response', 'Eval', 'Data', 'Instruction'])
    print(completions)

Empty DataFrame
Columns: [Response, Eval, Data, Instruction]
Index: []


In [37]:
prompt

[{'role': 'assistant', 'content': ''},
 {'role': 'user', 'content': 'Hello! How can I assist you today?'}]

In [39]:
for i in range(loops):
    completion = client.chat.completions.create(
        messages = prompt,
        #temperature = temperature,
        model = MODEL,
        )
    response = completion.choices[0].message.content

    data = {'Response': response, 'Eval': "", 'Data': {}, 'Instruction': prompt}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict()}
    completions = pd.concat([completions, data],ignore_index=True)
    completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [66]:
completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [44]:
completions = load("completions/gpt-3.5-turbo/1")

In [57]:
completions_old = load("completions/gpt-3.5-turbo/alphabet/6")

In [58]:
completions = pd.concat([completions, completions_old],ignore_index=True)

In [62]:
completions

,Response,Eval,Data,Instruction
0,Hello! How can I assist you today?,Best,{'Completion': {'id': 'chatcmpl-9wVhPaSkySWAXZ...,"[{'role': 'user', 'content': ''}]"
1,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wVhQtT4UIZdQZ...,"[{'role': 'user', 'content': ''}]"
2,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wVhRf25qD34kt...,"[{'role': 'user', 'content': ''}]"
3,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wVhSqokdZxkUS...,"[{'role': 'user', 'content': ''}]"
4,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wVhSePIGJrAxQ...,"[{'role': 'user', 'content': ''}]"
...,...,...,...,...
595,"Additionally, attending technology conferences...",,{'Completion': {'id': 'chatcmpl-9wmcebKVPpbyX7...,"[{'role': 'user', 'content': 'Stay up to date ..."
596,"Additionally, attending tech conferences and s...",,{'Completion': {'id': 'chatcmpl-9wmcguwws3Hw9E...,"[{'role': 'user', 'content': 'Stay up to date ..."
597,"\n\nAdditionally, attending tech events, confe...",,{'Completion': {'id': 'chatcmpl-9wmcizkG3AqW8V...,"[{'role': 'user', 'content': 'Stay up to date ..."
598,"\n\nAdditionally, attending industry conferenc...",,{'Completion': {'id': 'chatcmpl-9wmckmUh1b7xo4...,"[{'role': 'user', 'content': 'Stay up to date ..."


In [24]:
prompt = completions.iloc[n]["Instruction"]

In [25]:
prompt

[{'role': 'assistant', 'content': ''},
 {'role': 'user', 'content': 'Hello! How can I assist you today?'}]

In [65]:
n = 0
completions.iloc[n]

Response                      Hello! How can I assist you today?
Eval                                                      Common
Data           {'Completion': {'id': 'chatcmpl-9wVhPaSkySWAXZ...
Instruction                    [{'role': 'user', 'content': ''}]
Name: 0, dtype: object

In [64]:
tag = "Common"
completions.loc[n,"Eval"] = tag
completions.to_json(os.path.join(df_path, file), lines=True, orient="records")
selected_response = completions[completions["Eval"].apply(lambda x: x.startswith(tag))].iloc[0]["Response"]
if chat:
    messages = copy.deepcopy(completions[completions["Eval"].apply(lambda x: x.startswith(tag))].iloc[0]["Instruction"])
    messages.append({"role": "assistant", "content": selected_response_response})
    prompt = transform_roles(messages)
else:
    prompt = [{"role": "user", "content": selected_response}]
print(prompt)

OSError: Cannot save file into a non-existent directory: '/Users/janvotava/Desktop/openai/completions/gpt-4o-2024-08-06/chat/temperature-02'